In [81]:
# import dependencies
import pandas as pd
from random import sample
import numpy as np
import requests
from pprint import pprint
#from ..\\apikey.py import weatherkey
# import API key from file

In [93]:
# generate a list 500 random coordinate pairs that are well distributed across the coordinate range


longitudes = np.array(sample(range(-18000,18001),k=500))/100
latitudes = np.array(sample(range(-9000,9001),k=500))/100
print(longitudes)
print(latitudes)
# import citipy, create a list of cities from the random coordinate pairs

[ -34.96  116.04   78.55   78.28 -105.58    8.81 -161.33  171.37   45.02
 -142.53  111.54  -10.03  150.69   27.69  135.34   83.18 -171.4    80.55
   -5.42  -56.33 -155.75    2.    101.77  -39.1  -175.37   28.72  148.25
 -135.34  159.88   -5.19   83.22  -90.38 -160.62  109.44 -168.77 -179.77
  170.97   88.85   62.24  -19.97  116.55   96.38   13.79   17.09  126.32
   52.46 -170.25   73.52  155.74 -108.04  162.14 -105.72  169.42 -131.5
 -111.78    8.07  -69.28 -125.85  -97.94  115.1  -111.41  142.53 -100.1
    5.07   47.02  -24.03  154.61  143.84  110.11  -72.23  160.94 -149.45
  -27.56   72.32 -103.34   54.62  141.87   -4.87   84.86  105.96  -95.11
  176.59 -140.35 -178.74   59.49 -135.37   22.62 -130.5   108.98  175.22
  -89.16    8.47 -136.1   -77.55  152.17  131.1    70.01  -87.89  140.9
 -106.01    6.91   91.11  -26.52   23.67  -47.66   27.94    2.56   43.11
 -140.1   -40.62  133.54 -116.77   36.12  -27.64  -17.51   27.15  102.65
 -104.52 -159.27   83.01  -94.27  110.08  -39.53  -94.

In [77]:
print(latpop[0],latpop[1],latpop[-1],latpop[-2])
print(longpop[0],longpop[1],longpop[-1],longpop[-2])

-90.0 -89.99 90.0 89.99
-180.0 -179.99 180.0 179.99


In [10]:
#set up loop for each city in citylist
    # make an api call to openweathermap's API
    # return lat, long, temperatureF cloudiness%, Humidity%, windspeedmph
    #apendd data for each city to an existing list
    #print city name and number to a log file

#save data to a csv file

In [ ]:

#for each variable in list
    #plot scatter plot vs latitude
    #export each plot to a png file
    #if lat >o
        # plot all possible scatter plots on new plots
        #run linear regression for each plot
        #add regression results to the plot
        #export plot to png file
    #else
        #do the same as above on separate plots
        